In [1]:
import os
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
import pickle
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd
import re
import io
import json

In [2]:
def preprocess_txt(line, mode='q'): # mode='q'; mode='a' - for questions and answers
    txt = str(line)
    txt = re.split(r"[?!.]", txt)[0] # Берем только по одномк предложению
    txt = re.sub("<.{,3}>", " ", txt) # Убираем разметку
    txt = re.sub(r"[^а-яА-ЯёЁa-zA-Z,]+", " ", txt) # Заменяем на пробел все кроме нужных символов
    txt = txt.lower()
    txt = re.sub(r"([?.!,])", r" \1 ", txt) # Пробелы вокруг пунктуации
    txt = re.sub("\s{2,}", " ", txt) # убираем лишние пробелы
    
    if mode=='q':
        txt = re.sub("\sне\s", " не", txt)
        txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]  # if word not in sw
    elif mode=='a':
        txt = [word for word in txt.split()]
    else:
        assert mode in ['a', 'q'], "Mode error"
        
    return txt

In [3]:
#assert False

#Small preprocess of the answers

question = None
written = False
"""
with io.open("../data/prepared_answers.txt", "w", encoding='utf-8') as fout:
    with io.open("../data/Otvety.txt", "r", encoding='utf-8') as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue
                """

'\nwith io.open("../data/prepared_answers.txt", "w", encoding=\'utf-8\') as fout:\n    with io.open("../data/Otvety.txt", "r", encoding=\'utf-8\') as fin:\n        for line in tqdm_notebook(fin):\n            if line.startswith("---"):\n                written = False\n                continue\n            if not written and question is not None:\n                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))\n                written = True\n                question = None\n                continue\n            if not written:\n                question = line.strip()\n                continue\n                '

In [4]:
# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with io.open("../data/Otvety.txt", "r", encoding='utf-8') as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 200000:
            break

C:\Users\Eugene\anaconda3\lib\site-packages\ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


In [5]:
"""
sentences = [i for i in sentences if len(i) > 2]
modelFT = FastText(sentences=sentences, size=100, min_count=1, window=5)
modelFT.save("ft_model")"""

'\nsentences = [i for i in sentences if len(i) > 2]\nmodelFT = FastText(sentences=sentences, size=100, min_count=1, window=5)\nmodelFT.save("ft_model")'

In [6]:
modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with io.open("../data/prepared_answers.txt", "r", encoding='utf-8') as f:
    for line in tqdm_notebook(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT:
                vector_ft += modelFT[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1

        if counter > 200000:
            break
        
ft_index.build(10)
ft_index.save('speaker.ann')

C:\Users\Eugene\anaconda3\lib\site-packages\ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


C:\Users\Eugene\anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  from ipykernel import kernelapp as app
C:\Users\Eugene\anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


True

In [7]:
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('speaker.ann') 

True

In [8]:
with open("index_speaker.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [9]:
ft_index.get_nns_by_vector(np.zeros(100), 2)

[8, 1267]

In [10]:
shop_data = pd.read_csv("ProductsDataset.csv")

# shop_data['title'] + " " + shop_data["descrirption"] + " " + shop_data["properties"]
shop_data['text'] = shop_data['title'] + " " + shop_data["properties"]
shop_data['text'] = shop_data['text'].apply(lambda x: preprocess_txt(str(x)))

shop_data['info'] = shop_data['title'].apply(lambda x: preprocess_txt(str(x)))
drop_words = ['размер', 'цвет', 'детские', 'разм']
shop_data['info'] = shop_data['info'].apply(lambda x: [word for word in x if word not in drop_words])

shop_data.head()

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text,info
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, orby, detskie, razmer, rost, ,]","[юбка, детский, orby]"
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, zhenskaya, odezhda, tzvet, зелёный,...",[ботильон]
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, zhenskaya, odezhda, dzhinsy, bryuki, t...",[брюки]
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, detskie, pol, девочк...","[продать, детский, шапка]"
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, zhenskaya, odezhda, tzvet, синий, ,, ...",[блузка]


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

vectorizer = TfidfVectorizer(ngram_range=(1, 2))

In [12]:
# with open("index_speaker.pkl", "rb") as f:
#     index_map = pickle.load(f)

In [13]:
idxs = set(np.random.randint(0, len(index_map), len(shop_data)))
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in idxs]
positive_texts = [" ".join(val) for val in shop_data['text'].values]

In [14]:
dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, stratify=labels,
                                                    random_state=13)

In [16]:
x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

# lr = LogisticRegression().fit(x_train_vec, y_train)
lgbm_model = LGBMClassifier(num_leaves=31, 
                            learning_rate=0.05, 
                            n_estimators=1000,
                            reg_alpha=2,
                            reg_lambda=2,)
lr = lgbm_model.fit(x_train_vec, y_train)

In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.9701459693391037

In [18]:
with open("lr.pkl", "wb") as f:
    pickle.dump(lr, f)

In [19]:
with open("vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(X_train)

In [21]:
np.mean(tfidf_vect.idf_)

10.76517248320543

In [22]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}
idfs

{'включиться': 10.520248603186129,
 'ускоритель': 11.213395783746074,
 'мокасин': 11.213395783746074,
 'zhenskaya': 10.80793067563791,
 'odezhda': 8.96210398513958,
 'obuv': 11.213395783746074,
 'tip': 11.213395783746074,
 'кед': 11.213395783746074,
 'sezon': 11.213395783746074,
 'лето': 10.80793067563791,
 'tzvet': 10.29710505187192,
 'синий': 10.114783495077965,
 'razmer': 11.213395783746074,
 'николай': 10.520248603186129,
 'соболев': 11.213395783746074,
 'молодая': 11.213395783746074,
 'иметь': 11.213395783746074,
 'большой': 11.213395783746074,
 'популярность': 11.213395783746074,
 'интернет': 11.213395783746074,
 'дача': 10.29710505187192,
 'свой': 11.213395783746074,
 'сила': 11.213395783746074,
 'помощь': 10.80793067563791,
 'степлера': 11.213395783746074,
 'щвейной': 11.213395783746074,
 'машинка': 9.42163631451802,
 'правильно': 10.80793067563791,
 'сын': 11.213395783746074,
 'парткаха': 11.213395783746074,
 'мудь': 11.213395783746074,
 'звенеть': 11.213395783746074,
 'палитр

In [23]:
with open("idfs.pkl", "wb") as f:
    pickle.dump(idfs, f)

In [24]:
ft_index_shop = annoy.AnnoyIndex(100 ,'angular')

midf = np.mean(tfidf_vect.idf_)

index_map_shop = {}
counter = 0

for i in tqdm_notebook(range(len(shop_data))):
    n_ft = 0
    index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"], shop_data.loc[i, "info"])
    vector_ft = np.zeros(100)
    for word in shop_data.loc[i, "text"]:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_shop.add_item(counter, vector_ft)
    counter += 1

ft_index_shop.build(10)
ft_index_shop.save('shop.ann')

C:\Users\Eugene\anaconda3\lib\site-packages\ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


C:\Users\Eugene\anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  del sys.path[0]
C:\Users\Eugene\anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


True

In [25]:
with open("midf.pkl", "wb") as f:
    pickle.dump(midf, f)

In [26]:
with open("index_shop.pkl", "wb") as f:
    pickle.dump(index_map_shop, f)

In [27]:
def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [28]:
ft_index_shop.get_nns_by_vector(np.zeros(100), 1, include_distances=True)

([1505], [1.4142135381698608])

In [29]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

modelFT = FastText.load("ft_model")

ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('speaker.ann') 

ft_index_shop = annoy.AnnoyIndex(100, 'angular')
ft_index_shop.load('shop.ann') 

True

In [30]:
with open("index_speaker.pkl", "rb") as f:
    index_map = pickle.load(f)
    
with open("index_shop.pkl", "rb") as f:
    index_map_shop = pickle.load(f)

with open("vectorizer.pkl", "rb") as f:
    vectorizer = pickle.load(f)    
    
with open("idfs.pkl", "rb") as f:
    idfs = pickle.load(f)

with open("midf.pkl", "rb") as f:
    midf = pickle.load(f)
    
with open("lr.pkl", "rb") as f:
    lr = pickle.load(f)

In [31]:
message = 'синее платье'

In [32]:
input_txt = preprocess_txt(message)
vect = vectorizer.transform([" ".join(input_txt)])
prediction = lr.predict(vect)
result = []

if prediction[0] == 1:
    vect_ft = embed_txt(input_txt, idfs, midf)
    ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 2000)
    
    for item in ft_index_shop_val:
        title, image, info = index_map_shop[item]
        if len(set(info).intersection(set(input_txt))) > 1:
            result.insert(0, [title, image])
        elif len(set(info).intersection(set(input_txt))) > 0:
            result.append([title, image])
            
    for title, image in result[:5]:
        text="title: {} image: {}".format(title, image)
        print(text)
            
vect_ft = embed_txt(input_txt, {}, 1)
ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
if distances[0] > 0.2:
    print(distances[0])
    text="Моя твоя не понимать"
    print(text)
else:
    text=index_map[ft_index_val[0]]
    print(text)

title: Платье синее image: http://cache3.youla.io/files/images/360_360/5a/7f/5a7f3c07f567953cce201512.jpg
title: Платье из натуральной кожи питона image: http://cache3.youla.io/files/images/360_360/59/d1/59d1fcf6aaab2886684b6713.jpg
title: Лёгкое платье разлетайка image: http://cache3.youla.io/files/images/360_360/5b/d5/5bd5bc65eef1410ba05c99d5.jpg
title: Коктейльный платья Аренда image: http://cache3.youla.io/files/images/360_360/5a/87/5a87dfc9f20263b53840a432.jpg
title: Платье image: http://cache3.youla.io/files/images/360_360/58/7f/587fc317cd3022f864ce2e46.jpg
<p> Ответ на вопрос-цинк<br></p>. 



C:\Users\Eugene\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
C:\Users\Eugene\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
